In [1]:
import sys , os
import pandas as pd
import numpy as np

In [2]:
sys.path.append(os.path.join(os.path.abspath('..')))
# Import modules
from src import data_loader as dl
from scripts.Modelling import preprocess_data, split_the_data,define_models, define_hyperparameter_grids, perform_grid_search,evaluate_best_models,save_best_models

2025-03-09 15:29:00,092 - INFO - Directory already exists: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data
2025-03-09 15:29:00,100 - INFO - Directory already exists: c:\Users\HP\Documents\Dagii\Credit-scoring-model\src\../logs


## Step 1: Load Data

In [3]:
#import data
data = dl.load_data("transformed_data_credit_scoring.csv")
data.head()

2025-03-09 15:29:30,186 - INFO - Attempting to load data from: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data\transformed_data_credit_scoring.csv
2025-03-09 15:29:31,065 - INFO - Successfully loaded data from: c:\Users\HP\Documents\Dagii\Credit-scoring-model\data\transformed_data_credit_scoring.csv


,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,...,RFMS_score_binned_WOE,ProviderId_WOE,ProviderId_WOE.1,ProductId_WOE,ProductId_WOE.1,ProductCategory_WOE,ProductCategory_WOE.1,ChannelId_ChannelId_2,ChannelId_ChannelId_3,ChannelId_ChannelId_5
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,5,1,0,...,-0.071002,-0.413610,-0.413610,0.304295,0.304295,0.109343,0.109343,False,True,False
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,3,19,2,...,-0.071002,0.118382,0.118382,-0.154972,-0.154972,-0.057930,-0.057930,True,False,False
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,5,0,0,...,-0.071002,-0.413610,-0.413610,0.214869,0.214869,0.109343,0.109343,False,True,False
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,0,11,8,...,-0.071002,0.481371,0.481371,-0.361181,-0.361181,-0.427382,-0.427382,False,True,False
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,3,19,2,...,-0.071002,0.118382,0.118382,-0.154972,-0.154972,-0.057930,-0.057930,True,False,False


## Step 2: Preprocess Data
Data preprocessing is a critical step to prepare the raw data for modeling. This involves cleaning the data, handling missing values, encoding categorical variables, and extracting useful features.

In [5]:
# Preprocess the data using a predefined function
data = preprocess_data(data)
data.head()  

2025-03-09 15:29:38,155 - INFO - Starting data preprocessing
2025-03-09 15:29:38,156 - INFO - No duplicate columns found.
2025-03-09 15:29:38,159 - INFO - No unnecessary columns to drop.
2025-03-09 15:29:38,176 - INFO - No missing values found.
2025-03-09 15:29:38,178 - INFO - No categorical columns to encode.
2025-03-09 15:29:38,180 - INFO - 'TransactionStartTime' column not found. Skipping datetime feature extraction.
2025-03-09 15:29:38,181 - INFO - Data preprocessing completed successfully


,ProviderId,ProductCategory,Amount,Value,PricingStrategy,FraudResult,Total_Transaction_Amount,Average_Transaction_Amount,Transaction_Count,Std_Transaction_Amount,...,ProductId_17,ProductId_18,ProductId_19,ProductId_20,ProductId_21,ProductId_22,TransactionHour,TransactionDay,TransactionMonth,TransactionWeekday
0,5,0,-0.046371,-0.072291,2,0,0.170118,-0.067623,-0.311831,-0.167016,...,False,False,False,False,False,False,2,15,11,3
1,3,2,-0.054643,-0.080251,2,0,0.170118,-0.067623,-0.311831,-0.167016,...,False,False,True,False,False,False,2,15,11,3
2,5,0,-0.050426,-0.076352,2,0,0.165122,-0.072568,-0.444993,-0.201209,...,False,False,False,False,False,False,2,15,11,3
3,0,8,0.107717,0.096648,2,0,0.175567,-0.008155,-0.404020,-0.008243,...,False,False,False,False,False,False,3,15,11,3
4,3,2,-0.059704,-0.075183,2,0,0.175567,-0.008155,-0.404020,-0.008243,...,False,False,True,False,False,False,3,15,11,3


## Step 3: Split Data
To evaluate the model's performance on unseen data, i split the preprocessed dataset into training and testing sets. Typically, this is done with an 80-20 

In [6]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = split_the_data(data)
print(f"Training set shape: {X_train.shape}, Testing set shape: {X_test.shape}")

2025-03-09 15:29:48,699 - INFO - Starting data splitting
2025-03-09 15:29:48,796 - INFO - Data split into training and testing sets
Training set shape: (76529, 53), Testing set shape: (19133, 53)


## Step 4: Define Models and Hyperparameter Grids
Here,define the machine learning models to be trained and their hyperparameter grids for tuning. Common models for credit scoring include Logistic Regression (simple and interpretable) and Random Forest (powerful for non-linear patterns).

In [7]:
# Define the models and their hyperparameter grids
models = define_models()
param_grids = define_hyperparameter_grids()

2025-03-09 15:29:54,879 - INFO - Defining machine learning models
2025-03-09 15:29:54,881 - INFO - Models defined: LogisticRegression (with scaling), RandomForest
2025-03-09 15:29:54,883 - INFO - Defining hyperparameter grids
2025-03-09 15:29:54,883 - INFO - Hyperparameter grids defined for LogisticRegression and RandomForest


## Step 5: Perform Grid Search
Grid search is used to systematically test combinations of hyperparameters and find the best-performing configuration for each model. This step uses cross-validation to ensure robustness.

In [ ]:
# Perform grid search to optimize model hyperparameters
best_models = perform_grid_search(models, param_grids, X_train, y_train)

2025-03-09 15:30:00,658 - INFO - Starting Grid Search for hyperparameter tuning
2025-03-09 15:30:00,659 - INFO - Performing Grid Search for LogisticRegression


## Step 6: Evaluate Best Models
After tuning,  evaluate the best models on the test set to assess their real-world performance. Key metrics include accuracy, precision, recall, F1 score, and ROC-AUC, which are especially important for imbalanced datasets like credit scoring.

In [ ]:
# Evaluate the best models on the test set
results = evaluate_best_models(best_models, X_test, y_test)

## Step 7: Save Best Models
Finally, we save the tuned models to disk for future use, such as deployment in a production environment or further analysis.

In [ ]:
# Save the best models to disk
save_best_models(best_models)